In [1]:
# 형법 pdf
file_path = './dataset/criminal-law.pdf'

# Langchain의 PyPDFLoader를 이용
from langchain_community.document_loaders import PyPDFLoader
import re
from langchain_core.documents import Document

# PDF 로더 객체 생성
loader = PyPDFLoader(file_path)

# pdf의 각 페이지를 저장
pages = []

# 비동기 방식으로 pdf 페이지 로드
async for page in loader.alazy_load():
    pages.append(page)

In [2]:
# 맨앞 두 페이지는 쓸모없어서 버림
pages = pages[2:]

# 페이지 내용을 하나의 문자열로
full_text = "\n".join(page.page_content for page in pages)

In [3]:
# 문서 구조 추출을 위한 정규식 패턴
edition_pattern = re.compile(r'(제\d+편 [^\n]+)')
chapter_pattern = re.compile(r'(제\d+장 [^\n]+)')

# "장" 기준으로 분할
split_parts = re.split(chapter_pattern, full_text)

# 현재 편을 추적하기 위한 변수
current_edition = ""

# 첫 부분 처리 (첫 편/장 구조 파악)
chunks = []
first_content = split_parts[0].strip()
if first_content:
    # 편 정보 추출
    edition_match = edition_pattern.search(first_content)
    if edition_match:
        current_edition = edition_match.group(1)
    
    # 첫 부분을 문서로 생성
    first_page_meta = pages[0].metadata if pages else {}
    first_page_meta["edition"] = current_edition
    chunks.append(Document(page_content=first_content, metadata=first_page_meta))

# 이후 각 장별로 처리
for i in range(1, len(split_parts), 2):
    chapter_title = split_parts[i].strip()
    content = split_parts[i+1].strip() if i+1 < len(split_parts) else ""
    
    # 이 장의 내용에서 편 정보가 변경되었는지 확인
    edition_match = edition_pattern.search(content)
    if edition_match:
        current_edition = edition_match.group(1)
    
    metadata = {
        "chapter": chapter_title,
        "edition": current_edition
    }
    
    chunk_text = chapter_title + "\n" + content
    chunks.append(Document(page_content=chunk_text, metadata=metadata))

In [4]:
print(f"Total chunks created: {len(chunks)}")
if chunks:
    print("\n--- First Chunk ---")
    print(f"Metadata: {chunks[0].metadata}")
    print(f"Content:\n{chunks[0].page_content[:500]}...") # Print first 500 chars

if len(chunks) > 1:
    print("\n--- Second Chunk ---")
    print(f"Metadata: {chunks[1].metadata}")
    print(f"Content:\n{chunks[1].page_content[:500]}...") # Print first 500 chars

Total chunks created: 47

--- First Chunk ---
Metadata: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2013-11-08T14:53:09+09:00', 'moddate': '2013-11-08T14:53:09+09:00', 'source': './dataset/criminal-law.pdf', 'total_pages': 50, 'page': 2, 'page_label': '3', 'edition': '제1편 총칙'}
Content:
법제처                                                            3                                                       국가법령정보센터
「형법」
 
                    제1편 총칙...

--- Second Chunk ---
Metadata: {'chapter': '제1장 형법의 적용범위', 'edition': '제1편 총칙'}
Content:
제1장 형법의 적용범위
제1조(범죄의 성립과 처벌) ①범죄의 성립과 처벌은 행위 시의 법률에 의한다.
②범죄 후 법률의 변경에 의하여 그 행위가 범죄를 구성하지 아니하거나 형이 구법보다 경한
때에는 신법에 의한다.
③재판확정 후 법률의 변경에 의하여 그 행위가 범죄를 구성하지 아니하는 때에는 형의 집행
을 면제한다.
 
제2조(국내범) 본법은 대한민국영역 내에서 죄를 범한 내국인과 외국인에게 적용한다.
 
제3조(내국인의 국외범) 본법은 대한민국영역 외에서 죄를 범한 내국인에게 적용한다.
 
제4조(국외에 있는 내국선박 등에서 외국인이 범한 죄) 본법은 대한민국영역 외에 있는 대한민
국의 선박 또는 항공기 내에서 죄를 범한 외국인에게 적용한다.
 
제5조(외국인의 국외범) 본법은 대한민국영역 외에서 다음에 기재한 죄를 범한 외국인에게 적용
한다.
1. 내란